In [1]:
import arviz as az
import pymc3 as pm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from scipy import stats
from scipy.special import expit as logistic

warnings.simplefilter(action="ignore", category=FutureWarning)

RANDOM_SEED = 1234567

az.style.use("arviz-darkgrid")

In [2]:
UCB_data = '../../data/ucbadmit.csv'

In [3]:
df = pd.read_csv(UCB_data)
df.head()

,dept,applicant.gender,admit,reject,applications
1,A,male,512,313,825
2,A,female,89,19,108
3,B,male,353,207,560
4,B,female,17,8,25
5,C,male,120,205,325


In [4]:
df['applications_by_dept'] = df.groupby('dept')['applications'].transform('sum')

In [5]:
df['fraction_admitted'] = df['admit'] / df['applications']
df['frac_applicants'] = df['applications'] / df['applications_by_dept']
df

,dept,applicant.gender,admit,reject,applications,applications_by_dept,fraction_admitted,frac_applicants
1,A,male,512,313,825,933,0.620606,0.884244
2,A,female,89,19,108,933,0.824074,0.115756
3,B,male,353,207,560,585,0.630357,0.957265
4,B,female,17,8,25,585,0.680000,0.042735
5,C,male,120,205,325,918,0.369231,0.354031
6,C,female,202,391,593,918,0.340641,0.645969
7,D,male,138,279,417,792,0.330935,0.526515
8,D,female,131,244,375,792,0.349333,0.473485
9,E,male,53,138,191,584,0.277487,0.327055
10,E,female,94,299,393,584,0.239186,0.672945


## Model I

We wish to model the probability of admission, given the gender of the applicant. In the first pass, we ignore the department where admissions are being sought. 

The model is as follows:
\begin{align*}
    A_i                    & \sim \text{Binomial} (N_i, p_i) \\
    \text{logit}(p_i)      & \sim \alpha_{\text{gender}[i]}  \\
    \alpha_{\text{male}}   & \sim \text{Normal}(0, 1.5)      \\
    \alpha_{\text{female}} & \sim \text{Normal}(0, 1.5)
\end{align*}

Here $N_i$ is the number of applicants in row $i$ and $\alpha$ is an intercept term that may vary by gender. 

In [6]:
gender_ids, gender = pd.factorize(df['applicant.gender'])

with pm.Model() as m_11_7:
    a = pm.Normal('a', mu=0, sigma=1.5, shape=df['applicant.gender'].nunique())
    gender_id = pm.intX(pm.Data('gender_id', gender_ids))
    p = pm.Deterministic('p', pm.math.invlogit(a[gender_id]))
    
    admit = pm.Binomial('admit', n=df['applications'], p=p, observed=df['admit'])
    
    trace_11_7 = pm.sample(2000, tune=2000, return_inferencedata=False, random_seed=RANDOM_SEED)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a]


Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 4 seconds.


In [7]:
az.summary(trace_11_7, var_names=['a'], round_to=2, hdi_prob=0.89)

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a[0],-0.22,0.04,-0.28,-0.16,0.0,0.0,6881.64,5693.70,1.0
a[1],-0.83,0.05,-0.91,-0.75,0.0,0.0,6387.99,5202.95,1.0


In [8]:
gender_ids, gender

(array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]),
 Index(['male', 'female'], dtype='object'))